In [ ]:
import SimpleITK as sitk

def bspline_registration(fixed_image, moving_image, grid_physical_spacing=(20.0, 20.0, 20.0)):
    """
    Perform non-rigid B-spline registration.
    
    Args:
        fixed_image (SimpleITK.Image): Reference image.
        moving_image (SimpleITK.Image): Image to deform.
        grid_physical_spacing (tuple): Spacing for B-spline grid.

    Returns:
        SimpleITK.Image: Deformed/registered image.
    """
    transform_domain_mesh_size = [int(fixed_image.GetSize()[i] * fixed_image.GetSpacing()[i] / grid_physical_spacing[i]) for i in range(3)]
    transform = sitk.BSplineTransformInitializer(fixed_image, transform_domain_mesh_size)

    registration_method = sitk.ImageRegistrationMethod()
    registration_method.SetMetricAsMattesMutualInformation(50)
    registration_method.SetOptimizerAsLBFGSB(gradientConvergenceTolerance=1e-5, numberOfIterations=100)
    registration_method.SetInitialTransform(transform, inPlace=False)
    registration_method.SetInterpolator(sitk.sitkLinear)

    final_transform = registration_method.Execute(fixed_image, moving_image)
    registered_image = sitk.Resample(moving_image, fixed_image, final_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())

    return registered_image
